# k8s存储类

## 参考文献

* https://www.cnblogs.com/panwenbin-logs/p/12196286.html
* https://kubernetes.io/zh/docs/tasks/administer-cluster/change-default-storage-class/

## 存储类原理

![](https://bj.bcebos.com/ipic/k8s存储类.png)

## 实践

创建NFS服务器：

参考文献：*NFS服务器 notebook* 

实践：

`nfs-rbac.yaml`

```yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: nfs-client-provisioner
  namespace: default
---
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: nfs-client-provisioner-runner
rules:
  - apiGroups: [""]
    resources: ["persistentvolumes"]
    verbs: ["get", "list", "watch", "create", "delete"]
  - apiGroups: [""]
    resources: ["persistentvolumeclaims"]
    verbs: ["get", "list", "watch", "update"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["storageclasses"]
    verbs: ["get", "list", "watch"]
  - apiGroups: [""]
    resources: ["events"]
    verbs: ["create", "update", "patch"]
---
kind: ClusterRoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: run-nfs-client-provisioner
subjects:
  - kind: ServiceAccount
    name: nfs-client-provisioner
    # replace with namespace where provisioner is deployed
    namespace: default
roleRef:
  kind: ClusterRole
  name: nfs-client-provisioner-runner
  apiGroup: rbac.authorization.k8s.io
---
kind: Role
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: leader-locking-nfs-client-provisioner
    # replace with namespace where provisioner is deployed
  namespace: default
rules:
  - apiGroups: [""]
    resources: ["endpoints"]
    verbs: ["get", "list", "watch", "create", "update", "patch"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: leader-locking-nfs-client-provisioner
subjects:
  - kind: ServiceAccount
    name: nfs-client-provisioner
    # replace with namespace where provisioner is deployed
    namespace: default
roleRef:
  kind: Role
  name: leader-locking-nfs-client-provisioner
  apiGroup: rbac.authorization.k8s.io
```

`nfs-storage-class.yaml`

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: managed-nfs-storage
provisioner:  ${NFS_PROVISIONER_NAME} #这里的名称要和provisioner配置文件中的环境变量PROVISIONER_NAME保持一致
parameters:
  archiveOnDelete: "false"
```

* ` ${NFS_PROVISIONER_NAME}`存储提供着名字，要与`nfs-provisioner.yaml`对应的名字保持一致。

`nfs-provisioner.yaml`

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nfs-client-provisioner
  labels:
    app: nfs-client-provisioner
  namespace: default
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nfs-client-provisioner
  strategy:
    type: Recreate
  selector:
    matchLabels:
      app: nfs-client-provisioner
  template:
    metadata:
      labels:
        app: nfs-client-provisioner
    spec:
      serviceAccountName: nfs-client-provisioner
      containers:
        - name: nfs-client-provisioner
          image: quay.io/external_storage/nfs-client-provisioner:latest
          volumeMounts:
            - name: nfs-client-root
              mountPath: /persistentvolumes
          env:
            - name: PROVISIONER_NAME
              value: ${NFS_PROVISIONER_NAME}  #provisioner名称,请确保该名称与 nfs-StorageClass.yaml文件中的provisioner名称保持一致
            - name: NFS_SERVER
              value: ${NFS_SERVER_IP}
            - name: NFS_PATH  
              value: ${NFS_EXPORT_DIR}
      volumes:
        - name: nfs-client-root
          nfs:
            server: ${NFS_SERVER_IP}
            path: ${NFS_EXPORT_DIR}
```

* `${NFS_SERVER_IP}` NFS服务器IP；
* `${NFS_EXPORT_IDR}` NFS服务器导出目录；
* `${NFS_PROVISIONER_NAME` 存储提供者名字，要与`nfs-storage-class.yaml`对应的名字保持一致。

设置默认存储类

```shell
# 查看存储类
kubectl get storageclass;

# 设置默认存储类
kubectl patch storageclass <storage-class-name> -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"true"}}}'
```

所有的k8s集群节点安装nfs客户端

```shell
# 安装nfs客户端
sudo apt install -y nfs-common
```

>**重要：k8s所有节点都必须安装nfs客户端，否则可能导致很多无法创建PV，进到引起其他的异常，最终导致服务启动失败。**

## FAQ

Q：K8s创建nfs存储卷报错“ bad option; for several filesystems (e.g. nfs, cifs) you might need a /sbin/mount.”

A：k8s节点机器未安装nfs导致，`sudo apt install -y nfs-common`可以修复此问题。